In [1]:
import os

import torch

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import cooler

#import imputation function
from imputation.imputation import *

In [2]:
#config
resolution_rwr=100000
logscale=False
pad=1
std=1
rp=0.5 #restart probability to balance th information between global and local network structures
tol=0.01 #是什么意思
window_size=500000000
step_size=10000000
output_dist=500000000
min_cutoff=0
n_iters=20

## pos data

In [3]:
sv_list = pd.read_csv("./ref_data/sv_list.csv")
k562_list = sv_list[sv_list['Cell Line']=="K562"]
k562_list['diff'] = (k562_list['breakpoint2'] - k562_list['breakpoint1'])/1000

/tmp/ipykernel_249000/4095032312.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k562_list['diff'] = (k562_list['breakpoint2'] - k562_list['breakpoint1'])/1000


In [4]:
#选择chrom1=chrom2的
k562_list_intra = k562_list[k562_list['chrom1']==k562_list['chrom2']]
k562_list_intra.index = range(len(k562_list_intra))

In [5]:
#列出raw_data/K562/cooler/文件夹下所有的文件
file_list = os.listdir("./raw_data/scihic/K562/cooler/")
#读取所有的mcool文件
mcool_list = [i for i in file_list if i.endswith(".mcool")]
#读取所有文件名中不包含bulk的


In [117]:
def get_pos_submatrix_intra(clr,window,label_list,impute=False,resolution=100000
                            ,logscale=False,pad=1,std=1,rp=0.5,tol=0.01,window_size=500000000
                            ,step_size=10000000,output_dist=500000000,min_cutoff=0,n_iters=20):
    '''
    label_list都是intra的sv
    '''
    
    #找到有label对应的位置
    pos_data_list = []
    pos_label_list = []
    
    #遍历每一个sv的breakpoint
    for i in range(len(label_list)):
        #chrom1=chrom2
        chrom1 = label_list['chrom1'].iat[i]
        chrom2 = label_list['chrom2'].iat[i]
        #只取chromosome1的bin
        bin_table = clr.bins().fetch(chrom1)[:]
        #需要重新设置index
        bin_table.index = range(len(bin_table))
        
        breakpoint1 = label_list['breakpoint1'].iat[i]
        breakpoint2 = label_list['breakpoint2'].iat[i]
        string = label_list['strands'].iat[i]

        
        #找到对应的bin
        bin1 = bin_table[(bin_table['chrom']==chrom1) &(bin_table['start'] < breakpoint1 ) & (breakpoint1 < bin_table['end'] )]
        bin2 = bin_table[(bin_table['chrom']==chrom2) &(bin_table['start'] < breakpoint2 ) & (breakpoint2 < bin_table['end'] )]

        
        
        #breakpoint的坐标
        x_c = bin1.index[0]#得到的是索引
        y_c = bin2.index[0]
        

        x1 = x_c - int((window-1)/2)
        x2 = x_c + int((window-1)/2)

        
        y1 = y_c - int((window-1)/2)
        y2 = y_c + int((window-1)/2)
        
        
        #如果x1或y1小于0，就取0
        if x1 < 0:
            x1 = 0
            x2 = window-1
        if y1 < 0:
            y1 = 0
            y2 = window-1
        #如果x2或y2大于区域的长度，就取区域的长度
        max_length= clr.matrix(balance=False).fetch(chrom1).shape[0]

        if x2 > max_length:
            x2 = max_length-1
            x1 = max_length-window
        if y2 > max_length:
            y2 = max_length-1
            y1 = max_length-window
        
    
        if impute:
            #impute by rwr
            matrix = imputation_rwr(clr,chrom1,resolution,logscale,pad,std,rp,tol,window_size
                    ,step_size,output_dist,min_cutoff,n_iters)
            #将matrix转成array
            matrix = np.array(matrix)
            
        else:
            #raw matrix
            matrix = clr.matrix(balance=False)[:]
        #fetch submatrix
        submatrix = matrix[x1:x2+1, y1:y2+1]

        
        #pos_data_list中添加submatrix
        pos_data_list.append(submatrix)
        pos_label_list.append(string)

    return pos_data_list,pos_label_list



In [124]:
resolution = 100000
window = 21
cool_dir = "raw_data/scihic/K562/cooler/"
#读取所有的mcool文件
pos_data_list = []
pos_label_list = []

#按细胞做处理
for mc in mcool_list[1:3]:
    clr = cooler.Cooler(cool_dir+mc+"::/resolutions/"+str(resolution))
    sc_pos_data_list,sc_pos_label_list = get_pos_submatrix_intra(clr,window,k562_list_intra
                                                                ,True,resolution_rwr,
                                                                logscale,pad,std,rp,tol,
                                                                window_size,step_size,
                                                                output_dist,min_cutoff,n_iters) 
    

    pos_data_list.append(sc_pos_data_list)
    pos_label_list.append(sc_pos_label_list)

#fetch data from list    
pos_data_list = [item for sublist in pos_data_list for item in sublist]
pos_label_list = [item for sublist in pos_label_list for item in sublist]


pos_data_array = np.dstack(pos_data_list)
pos_data_array = np.rollaxis(pos_data_array,-1)

pos_label_list = np.array(pos_label_list)

#保存
np.save("input_data/pos_data_intra_imputation.npy",pos_data_array)
np.save("input_data/pos_label_intra_imputation.npy",pos_label_list)


In [123]:
pos_label_list

['+-',
 '+-',
 '+-',
 '-+',
 '--',
 '+-',
 '-+',
 '-+',
 '-+',
 '-+',
 '-+',
 '+-',
 '-+',
 '-+',
 '-+',
 '+-',
 '--',
 '+-',
 '-+',
 '+-',
 '-+',
 '++',
 '+-',
 '-+',
 '+-',
 '--',
 '+-',
 '+-',
 '-+',
 '-+',
 '-+',
 '+-',
 '+-',
 '++',
 '--',
 '-+',
 '+-',
 '+-',
 '+-',
 '-+',
 '--',
 '+-',
 '-+',
 '-+',
 '-+',
 '-+',
 '-+',
 '+-',
 '-+',
 '-+',
 '-+',
 '+-',
 '--',
 '+-',
 '-+',
 '+-',
 '-+',
 '++',
 '+-',
 '-+',
 '+-',
 '--',
 '+-',
 '+-',
 '-+',
 '-+',
 '-+',
 '+-',
 '+-',
 '++',
 '--',
 '-+']

In [68]:
#pos_data_list转成72*21*21的array


(72,)

In [71]:
pos_data_list[0].shape

(21, 21)

In [154]:
#比较一下imputation前后的数据吧
pos_data_list1 = np.load("input_data/pos_data_list.npy")
pos_label_list1 = np.load("input_data/pos_label_list.npy")



In [157]:
pos_data_list2 = np.load("input_data/pos_data_intra_imputation.npy")
pos_label_list2 = np.load("input_data/pos_label_intra_imputation.npy")

In [159]:
#pos_data_list1中非0个数
pos_data_list1[pos_data_list1!=0].shape

(56189,)

In [160]:
pos_data_list2[pos_data_list2!=0].shape

(601344,)

In [78]:
pos_data_list2

#把pos_data_list2转成和pos_data_list1一样的格式
pos_data_list2 = np.expand_dims(pos_data_list2, axis=1)

In [83]:
# pos_data_list2里面的每一个元素都是一个array，需要转成array
pos_data_list2 = np.array(pos_data_list2)

## neg data

In [127]:
#列出raw_data/K562/cooler/文件夹下所有的文件
gm12878_file_list = os.listdir("./raw_data/scihic/GM12878/cooler/")
#读取所有的mcool文件
gm12878_mcool_list = [i for i in gm12878_file_list if i.endswith(".mcool")]


### 和cancer cell的SV对应的区域的

In [5]:
def get_neg_submatrix_intra(clr,window,label_list,impute=False,resolution=100000
                            ,logscale=False,pad=1,std=1,rp=0.5,tol=0.01,window_size=500000000
                            ,step_size=10000000,output_dist=500000000,min_cutoff=0,n_iters=20):
    '''
    label_list都是intra的sv
    '''
    
    #找到有label对应的位置
    neg_data_list = []
    neg_label_list = []
    
    #遍历每一个sv的breakpoint
    for i in range(len(label_list)):
        #chrom1=chrom2
        chrom1 = label_list['chrom1'].iat[i]
        chrom2 = label_list['chrom2'].iat[i]
        #只取chromosome1的bin
        bin_table = clr.bins().fetch(chrom1)[:]
        #需要重新设置index
        bin_table.index = range(len(bin_table))
        
        breakpoint1 = label_list['breakpoint1'].iat[i]
        breakpoint2 = label_list['breakpoint2'].iat[i]
        
        #找到对应的bin
        bin1 = bin_table[(bin_table['chrom']==chrom1) &(bin_table['start'] < breakpoint1 ) & (breakpoint1 < bin_table['end'] )]
        bin2 = bin_table[(bin_table['chrom']==chrom2) &(bin_table['start'] < breakpoint2 ) & (breakpoint2 < bin_table['end'] )]

        #breakpoint的坐标
        x_c = bin1.index[0]#得到的是索引
        y_c = bin2.index[0]
        
        x1 = x_c - int((window-1)/2)
        x2 = x_c + int((window-1)/2)
        
        y1 = y_c - int((window-1)/2)
        y2 = y_c + int((window-1)/2)
        
        
        #如果x1或y1小于0，就取0
        if x1 < 0:
            x1 = 0
            x2 = window-1
        if y1 < 0:
            y1 = 0
            y2 = window-1
        #如果x2或y2大于区域的长度，就取区域的长度
        max_length= clr.matrix(balance=False).fetch(chrom1).shape[0]

        if x2 > max_length:
            x2 = max_length-1
            x1 = max_length-window
        if y2 > max_length:
            y2 = max_length-1
            y1 = max_length-window
        
    
        if impute:
            #impute by rwr
            matrix = imputation_rwr(clr,chrom1,resolution,logscale,pad,std,rp,tol,window_size
                    ,step_size,output_dist,min_cutoff,n_iters)
            #将matrix转成array
            matrix = np.array(matrix)
            
        else:
            #raw matrix
            matrix = clr.matrix(balance=False)[:]
        #fetch submatrix
        submatrix = matrix[x1:x2+1, y1:y2+1]

        
        #pos_data_list中添加submatrix
        neg_data_list.append(submatrix)
        neg_label_list.append("no")

    return neg_data_list,neg_label_list



In [14]:
resolution = 100000
window = 21
cool_dir = "raw_data/scihic/GM12878/cooler/"
#读取所有的mcool文件
neg_data_list = []
neg_label_list = []

#按细胞做处理
for mc in gm12878_mcool_list:
    clr = cooler.Cooler(cool_dir+mc+"::/resolutions/"+str(resolution))
    
    #取对应位置的
    sc_neg_data_list,sc_neg_label_list = get_neg_submatrix_intra(clr,window,k562_list_intra
                                                                ,True,resolution_rwr,
                                                                logscale,pad,std,rp,tol,
                                                                window_size,step_size,
                                                                output_dist,min_cutoff,n_iters) 
    
    neg_data_list.append(sc_neg_data_list)
    neg_label_list.append(sc_neg_label_list)

#fetch data from list    
neg_data_list = [item for sublist in neg_data_list for item in sublist]
neg_label_list = [item for sublist in neg_label_list for item in sublist]


neg_data_array = np.dstack(neg_data_list)
neg_data_array = np.rollaxis(neg_data_array,-1)
neg_label_list = np.array(neg_label_list)

#保存
np.save("input_data/gm12878_data_intra_imputation_correspond.npy",neg_data_array)
np.save("input_data/gm12878_label_intra_imputation_correspond.npy",neg_label_list)


: 

: 

In [117]:

def get_random_matrix_gm12878(num_sample,window,clr):
    rand_data_list = []
    rand_label_list = []
    for i in range(num_sample):
        whole_matrix = clr.matrix(balance=False)[:]
        x = np.random.randint(0,whole_matrix.shape[0]-window)
        y = np.random.randint(0,whole_matrix.shape[1]-window)
        submatrix = whole_matrix[x:x+window,y:y+window]
        rand_data_list.append(submatrix)
        label = "no"
        rand_label_list.append(label)
    return rand_data_list,rand_label_list


In [118]:
cool_dir = "raw_data/scihic/GM12878/cooler/"
resolution = 200000
window = 21
num = 5

neg12878_data_list = []
neg12878_label_list = []

for mc in gm12878_mcool_list:
    clr = cooler.Cooler(cool_dir+mc+"::/resolutions/"+str(resolution))
    sc_neg_data_list,sc_neg_label_list = get_random_matrix_gm12878(num,window,clr)
    neg12878_data_list.append(sc_neg_data_list)
    neg12878_label_list.append(sc_neg_label_list)

neg12878_data_list = [item for sublist in neg12878_data_list for item in sublist]
neg12878_label_list = [item for sublist in neg12878_label_list for item in sublist]
neg12878_data_list = np.array(neg12878_data_list)
neg12878_label_list = np.array(neg12878_label_list)
#数据类型
neg12878_data_list = neg12878_data_list.astype(np.float32)
#保存
np.save("input_data/neg12878_data_list.npy",neg12878_data_list)
np.save("input_data/neg12878_label_list.npy",neg12878_label_list)


In [127]:
#for K562
def determine_label(tmp_bin_x,tmp_bin_y,k562_list):
    
    label = "no"
    for i in range(len(tmp_bin_x)):
        chr1 = tmp_bin_x['chrom'].iat[i]
        start1 = tmp_bin_x['start'].iat[i]
        end1 = tmp_bin_x['end'].iat[i]
        for j in range(len(tmp_bin_y)):
            chr2 = tmp_bin_y['chrom'].iat[j]
            start2 = tmp_bin_y['start'].iat[j]
            end2 = tmp_bin_y['end'].iat[j]

            for m in range(len(k562_list)):
                chrom1 = k562_list['chrom1'].iat[m]
                chrom2 = k562_list['chrom2'].iat[m]
                breakpoint1 = k562_list['breakpoint1'].iat[m]
                breakpoint2 = k562_list['breakpoint2'].iat[m]
                string = k562_list['strands'].iat[m]
                if (chr1 == chrom1 and chr2==chrom2 and (breakpoint1 > start1 and breakpoint1<end1) and (breakpoint2 > start2 and breakpoint2<end2) ):
                    label = string
                    break
                elif (chr1 == chrom2 and chr2==chrom1 and (breakpoint1 > start2 and breakpoint1<end2) and (breakpoint2 > start1 and breakpoint2<end1)):
                    label = string
                    break
            if label != "no":
                break
        if label != "no":
            break
    return label

#从K562中获得
#很少，但吧
neg_data_list = []
neg_label_list = []


sample_x1=np.random.randint(7000, size=(50))
sample_y1=np.random.randint(7000, size=(50))
sample_x2 = sample_x1 + window
sample_y2 = sample_y1 + window

for x,y in zip(sample_x1,sample_y1):
    submatrix = matrix1[x:x+window, y:y+window]
    nozeron = np.count_nonzero(submatrix)
    tmp_bin_x = bin_table.loc[x:x+window,:]
    tmp_bin_y = bin_table.loc[y:y+window,:]
    #得到标签
    label = determine_label(tmp_bin_x,tmp_bin_y,k562_list)

    neg_data_list.append(submatrix)
    neg_label_list.append(label)        

### loop附近的

In [133]:
#读loop的数据
loop_path = "/share/home/mliu/sc_sv/raw_data/GM12878/GSE63525_GM12878_primary+replicate_HiCCUPS_looplist.txt.gz"
gm12878_loop_list = pd.read_csv(loop_path,sep="\t",compression='gzip')[['chr1','x1','x2','chr2','y1','y2']]


In [138]:
gm12878_loop_list['chr1'] = gm12878_loop_list['chr1'].apply(lambda x: "chr"+str(x))
gm12878_loop_list['chr2'] = gm12878_loop_list['chr2'].apply(lambda x: "chr"+str(x))

In [174]:

#随机选择其中的n个位点，取
def get_neg_loop_matrix(clr,window,loop_list,n_sample,impute,resolution_rwr,
                        logscale,pad,std,rp,tol,
                        window_size,step_size,
                        output_dist,min_cutoff,n_iters):
    
    neg_data_list = []
    neg_label_list = []
    #在loop_list中随机取n行
    sample_loop_list = loop_list.sample(n=n_sample)
    for i in range(len(sample_loop_list)):
        chrom1 = sample_loop_list['chr1'].iat[i]
        chrom2 = sample_loop_list['chr2'].iat[i]

        bin_table = clr.bins().fetch(chrom1)[:]
        bin_table.index = range(len(bin_table))

        pos1 = (sample_loop_list['x1'].iat[i] + sample_loop_list['x2'].iat[i])/2
        pos2 = (sample_loop_list['y1'].iat[i] + sample_loop_list['y2'].iat[i])/2

        #找到对应的bin

        bin1 = bin_table[(bin_table['start']<=pos1) & (bin_table['end']>=pos1)]
        bin2 = bin_table[(bin_table['start']<=pos2) & (bin_table['end']>=pos2)]

        #index用来fetch矩阵
        x_c = bin1.index[0]
        y_c = bin2.index[0]

        x1 = x_c - int((window-1)/2)
        x2 = x_c + int((window-1)/2)

        
        y1 = y_c - int((window-1)/2)
        y2 = y_c + int((window-1)/2)

        if x1 < 0:
            x1 = 0
            x2 = window-1
        if y1 < 0:
            y1 = 0
            y2 = window-1
        #如果x2或y2大于区域的长度，就取区域的长度
        max_length= clr.matrix(balance=False).fetch(chrom1).shape[0]

        if x2 > max_length:
            x2 = max_length-1
            x1 = max_length-window
        if y2 > max_length:
            y2 = max_length-1
            y1 = max_length-window
        
        if impute:
            #impute by rwr
            matrix = imputation_rwr(clr,chrom1,resolution,logscale,pad,std,rp,tol,window_size
                    ,step_size,output_dist,min_cutoff,n_iters)
            #将matrix转成array
            matrix = np.array(matrix)
            
        else:
            #raw matrix
            matrix = clr.matrix(balance=False)[:]
        #fetch submatrix
        submatrix = matrix[x1:x2+1, y1:y2+1]

        if(submatrix.shape[0]==window):
            neg_data_list.append(submatrix)
            neg_label_list.append("no")
    
    return neg_data_list,neg_label_list



In [175]:
resolution = 100000
window = 21
cool_dir = "raw_data/scihic/GM12878/cooler/"
n_sample = 10
#读取所有的mcool文件
neg_data_list = []
neg_label_list = []

#按细胞做处理
for mc in gm12878_mcool_list:
    clr = cooler.Cooler(cool_dir+mc+"::/resolutions/"+str(resolution))
    
    #取对应位置的
    sc_neg_data_list,sc_neg_label_list = get_neg_loop_matrix(clr,window,gm12878_loop_list,
                                                                n_sample
                                                                ,True
                                                                ,resolution_rwr,
                                                                logscale,pad,std,rp,tol,
                                                                window_size,step_size,
                                                                output_dist,min_cutoff,n_iters) 
    
    neg_data_list.append(sc_neg_data_list)
    neg_label_list.append(sc_neg_label_list)

#fetch data from list    
neg_data_list = [item for sublist in neg_data_list for item in sublist]
neg_label_list = [item for sublist in neg_label_list for item in sublist]

#FIXME:here
neg_data_array = np.dstack(neg_data_list)
neg_data_array = np.rollaxis(neg_data_array,-1)
neg_label_list = np.array(neg_label_list)

#保存
np.save("input_data/gm12878_data_intra_imputation_loop.npy",neg_data_array)
np.save("input_data/gm12878_label_intra_imputation_loop.npy",neg_label_list)


: 

: 

In [172]:
#去掉neg_data_list中shape为20*20的元素
neg_data_list_new = [item for item in neg_data_list if item.shape[0]==21]
neg_data_array = np.dstack(neg_data_list_new)
neg_data_array = np.rollaxis(neg_data_array,-1)
neg_label_list = np.array(neg_label_list)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 21 and the array at index 819 has size 20

In [2]:
import numpy as np
test = np.load("input_data/gm12878_data_intra_imputation_loop.npy")

In [4]:
test.shape

(3423, 21, 21)

### cancer cell line SV block附近的

In [ ]:
#TODO： write


## 整合数据

In [34]:
label_dic = {'++':1,'+-':2,'-+':3,'--':4,'no':0}

In [35]:
pos_data_no_impute = np.load("input_data/pos_data_intra.npy")
pos_label_no_impute = np.load("input_data/pos_label_intra.npy")

neg_data_no_impute_cor = np.load("input_data/gm12878_data_intra_correspond_noimpute.npy")
neg_label_no_impute_cor = np.load("input_data/gm12878_label_intra_correspond_noimpute.npy")

neg_data_no_impute_loop = np.load("input_data/gm12878_data_intra_loop_noimpute.npy")
neg_label_no_impute_loop = np.load("input_data/gm12878_label_intra_loop_noimpute.npy")


pos_data_impute = np.load("input_data/pos_data_intra_imputation.npy")
pos_label_impute = np.load("input_data/pos_label_intra_imputation.npy")

neg_data_impute_cor = np.load("input_data/gm12878_data_intra_imputation_correspond.npy")
neg_label_impute_cor = np.load("input_data/gm12878_label_intra_imputation_correspond.npy")

neg_data_impute_loop = np.load("input_data/gm12878_data_intra_imputation_loop.npy")
neg_label_impute_loop = np.load("input_data/gm12878_label_intra_imputation_loop.npy")

In [36]:
print("------raw------")
print("pos")
print(pos_data_no_impute.shape)
print(pos_label_no_impute.shape)
print("neg cor region")
print(neg_data_no_impute_cor.shape)
print(neg_label_no_impute_cor.shape)
print("neg loop region")
print(neg_data_no_impute_loop.shape)
print(neg_label_no_impute_loop.shape)

print("-----impute------")
print("pos")
print(pos_data_impute.shape)
print(pos_label_impute.shape)
print("neg cor region")
print(neg_data_impute_cor.shape)
print(neg_label_impute_cor.shape)
print("neg loop region")
print(neg_data_impute_loop.shape)
print(neg_label_impute_loop.shape)

------raw------
pos
(1807, 21, 21)
(1807,)
neg cor region
(1421, 21, 21)
(1421,)
neg loop region
(3430, 21, 21)
(3430,)
-----impute------
pos
(8989, 21, 21)
(8989,)
neg cor region
(8214, 21, 21)
(8214,)
neg loop region
(3353, 21, 21)
(3353,)


In [37]:
all_data_no_impute = np.concatenate((pos_data_no_impute,neg_data_no_impute_cor,neg_data_no_impute_loop),axis=0)
all_label_no_impute = np.concatenate((pos_label_no_impute,neg_label_no_impute_cor,neg_label_no_impute_loop),axis=0)

all_data_impute = np.concatenate((pos_data_impute,neg_data_impute_cor,neg_data_impute_loop),axis=0)
all_label_impute = np.concatenate((pos_label_impute,neg_label_impute_cor,neg_label_impute_loop),axis=0)

In [38]:
window = 21
print(all_data_no_impute.shape)
all_data_no_impute = all_data_no_impute.reshape(all_data_no_impute.shape[0],1,window,window)
print(all_data_no_impute.shape)

print(all_data_impute.shape)
all_data_impute = all_data_impute.reshape(all_data_impute.shape[0],1,window,window)
print(all_data_impute.shape)

(6658, 21, 21)
(6658, 1, 21, 21)
(20556, 21, 21)
(20556, 1, 21, 21)


In [40]:
all_label_no_impute_index = [label_dic[i] for i in all_label_no_impute]
all_label_impute_index = [label_dic[i] for i in all_label_impute]
print(np.unique(all_label_no_impute_index))
print(np.unique(all_label_impute_index))

In [42]:
np.save('input_data/data_impute.npy',all_data_impute)   
np.save('input_data/label_impute.npy',all_label_impute_index)

np.save('input_data/data_no_impute.npy',all_data_no_impute)
np.save('input_data/label_no_impute.npy',all_label_no_impute_index)